<a href="https://colab.research.google.com/github/OOJNIM/23fall_LB-C/blob/main/class2023Fall_1201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow gradio imageio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.7 MB/s eta 0:00:00


In [ ]:
# one feature regress
import gradio as gr
import imageio
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# Params
NUM_STEPS = 300
NUM_DATA = 60

# Data
np.random.seed(0)
input_data = np.linspace(-1, 1, NUM_DATA)
target_data = 2 * input_data + np.random.randn(*input_data.shape) * 0.27  # y = 2*x + noise

# Model
model = Sequential([Dense(1, input_dim=1)])
model.compile(optimizer='sgd', loss='mean_squared_error')

# Function to train the model for one step and return weights_matrix and output_data
def train_one_step(model, input_data, target_data):
    model.train_on_batch(input_data, target_data)
    weights_matrix = model.layers[0].get_weights()[0][0][0]
    output_data = model.predict(input_data)
    return weights_matrix, output_data

# Lists to store weights_matrix and output_data at each step
weights_over_steps = []
outputs_over_steps = []

# Functions for Plotting
def plot_weights_over_time(step):
    plt.figure()
    plt.plot(weights_over_steps[:step+1])
    plt.xlabel('step')
    plt.ylabel('weights_matrix')
    plt.title(f'Model Weights at Step: {step}')
    plt.savefig(f'1f_weights_step_{step}.png')
    plt.close()

def plot_regression_over_time(step):
    plt.figure()
    plt.scatter(input_data, target_data, color='blue')
    plt.plot(input_data, outputs_over_steps[step], color='red')
    plt.xlabel('input_data')
    plt.ylabel('target_data')
    plt.title(f'Regression Line at Step {step}')
    plt.savefig(f'1f_regression_step_{step}.png')
    plt.close()


# Manually training for each step
for step in range(NUM_STEPS):
    weights_matrix, output_data = train_one_step(model, input_data, target_data)
    weights_over_steps.append(weights_matrix)
    outputs_over_steps.append(output_data)
    plot_weights_over_time(step)
    plot_regression_over_time(step)

# For gradio: Create GIF for Weights Evolution
weights_images = [f'1f_weights_step_{i}.png' for i in range(NUM_STEPS)]
imageio.mimsave('1f_weights_evolution.gif', [imageio.imread(img) for img in weights_images], fps=4, loop=0)

# For gradio: Create GIF for Regression Evolution
regression_images = [f'1f_regression_step_{i}.png' for i in range(NUM_STEPS)]
imageio.mimsave('1f_regression_evolution.gif', [imageio.imread(img) for img in regression_images], fps=4, loop=0)

# For gradio: plain png Weights loader
def show_weights_image(step):
    return f'1f_weights_step_{step}.png'

# For gradio: plain png Regression loader
def show_regression_image(step):
    return f'1f_regression_step_{step}.png'

# Create a Blocks interface
with gr.Blocks() as demo:
    gr.Markdown("# 2D Model Training Visualization")

    with gr.Row():
        with gr.Column():
            step_slider = gr.Slider(0, NUM_STEPS - 1, step=1, label="Select Step")
            weight_image = gr.Image(label="Weights at Step")
            regression_image = gr.Image(label="Regression at Step")
            step_slider.change(fn=show_weights_image, inputs=step_slider, outputs=weight_image)
            step_slider.change(fn=show_regression_image, inputs=step_slider, outputs=regression_image)

        with gr.Column():
            gr.Image(value='1f_weights_evolution.gif', label="Weights Evolution (GIF)")
            gr.Image(value='1f_regression_evolution.gif', label="Regression Evolution (GIF)")

# Launch the interface
demo.launch()


In [ ]:
# two features regress
import gradio as gr
import imageio
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# Params
NUM_STEPS = 200
NUM_DATA = 60

# Data
np.random.seed(0)
x = np.linspace(-1, 1, NUM_DATA)
y = np.linspace(-1, 1, NUM_DATA)
X_1, X_2 = np.meshgrid(x, y)
Y = 2 * X_1 + 3 * X_2 + np.random.randn(*X_1.shape) * 0.27  # y = 2*x1 + 3*x2 + noise
input_data = np.column_stack([X_1.flatten(), X_2.flatten()])
target_data = Y.flatten()

# Model
model = Sequential([Dense(1, input_dim=2)])
model.compile(optimizer='sgd', loss='mean_squared_error')

# Function to train the model for one step and return weights_matrix and output_data
def train_one_step(model, input_data, target_data):
    model.train_on_batch(input_data, target_data)
    weights_matrix = model.layers[0].get_weights()[0]
    output_data = model.predict(input_data).reshape(X_1.shape)
    return weights_matrix, output_data

# Lists to store weights_matrix and output_data at each step
weights_over_steps = []
outputs_over_steps = []

# Functions for Plotting
def plot_weights_over_time(step):
    plt.figure()
    plt.plot([w[0] for w in weights_over_steps[:step + 1]])
    plt.plot([w[1] for w in weights_over_steps[:step + 1]])
    plt.xlabel('step')
    plt.ylabel('weights_matrix')
    plt.title(f'Model Weights at Step: {step}')
    plt.legend(['Weight 1', 'Weight 2'])
    plt.savefig(f'2f_weights_step_{step}.png')
    plt.close()

def plot_regression_over_time(step):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(X_1, X_2, target_data, color='blue')
    ax.plot_surface(X_1, X_2, outputs_over_steps[step], color='red', alpha=0.6)
    ax.set_xlabel('X_1')
    ax.set_ylabel('X_2')
    ax.set_zlabel('target_data')
    ax.set_title(f'Regression Plane at Step {step}')
    plt.savefig(f'2f_regression_step_{step}.png')
    plt.close()

# Manually training for each step
for step in range(NUM_STEPS):
    weights_matrix, output_data = train_one_step(model, input_data, target_data)
    weights_over_steps.append(weights_matrix)
    outputs_over_steps.append(output_data)
    plot_weights_over_time(step)
    plot_regression_over_time(step)

# For gradio: Create GIF for Weights Evolution
weights_images = [f'2f_weights_step_{i}.png' for i in range(NUM_STEPS)]
imageio.mimsave('2f_weights_evolution.gif', [imageio.imread(img) for img in weights_images], fps=4, loop=0)

# For gradio: Create GIF for Regression Evolution
regression_images = [f'2f_regression_step_{i}.png' for i in range(NUM_STEPS)]
imageio.mimsave('2f_regression_evolution.gif', [imageio.imread(img) for img in regression_images], fps=4, loop=0)

# For gradio: plain png Weights loader
def show_weights_image(step):
    return f'2f_weights_step_{step}.png'

# For gradio: plain png Regression loader
def show_regression_image(step):
    return f'2f_regression_step_{step}.png'

# Create a Blocks interface
with gr.Blocks() as demo:
    gr.Markdown("# 3D Model Training Visualization")

    with gr.Row():
        with gr.Column():
            step_slider = gr.Slider(0, NUM_STEPS - 1, step=1, label="Select Step")
            weight_image = gr.Image(label="Weights at Step")
            regression_image = gr.Image(label="Regression at Step")
            step_slider.change(fn=show_weights_image, inputs=step_slider, outputs=weight_image)
            step_slider.change(fn=show_regression_image, inputs=step_slider, outputs=regression_image)

        with gr.Column():
            gr.Image(value='2f_weights_evolution.gif', label="Weights Evolution (GIF)")
            gr.Image(value='2f_regression_evolution.gif', label="Regression Evolution (GIF)")

# Launch the interface
demo.launch()
